## Context of the Project

The Supermarket Sales project aims to analyze sales data from a supermarket to gain insights into customer behavior, sales trends, and inventory management. By leveraging data analysis and visualization techniques, the project seeks to answer key business questions and support decision-making processes.


# Datas Description

### Columns:

- **Invoice ID**: Unique identifier for each transaction.
- **Branch**: The branch of the supermarket where the transaction took place (A, B, or C).
- **City**: The city where the branch is located (e.g., New York, Los Angeles, Chicago).
- **Customer Type**: Type of customer (e.g., Member, Normal).
- **Gender**: Gender of the customer (Male, Female).
- **Product Line**: Category of products purchased (e.g., Health and beauty, Electronic accessories, Home and lifestyle, Sports and travel, Food and beverages, Fashion accessories).
- **Unit Price**: Price per unit of the product.
- **Quantity**: Number of units purchased.
- **Tax 5%**: Tax amount applied to the transaction (5% of the total price).
- **Total**: Total amount paid by the customer (including tax). 
- **Rewarded Points**: Loyalty points earned by the customer from the transaction.

In [ ]:
# Imports 

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


